First, need to parse xml files. Convert to csv to make easier to work with. 

In [40]:
import csv
import xml.etree.ElementTree as ET
    
def parsexml(xmlFile, fileName):
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    
    # open csv file to write to
    data = open(fileName, 'w')
    csvwriter = csv.writer(data)

    # get keys from object 
    head = list(root[0].attrib.keys())
    csvwriter.writerow(head)
    
    # write each line to csv file
    for line in root.findall('row'):
        lineValues = []
        for key in head:
            #if key doesn't exist, just put empty for that key's value
            lineValues.append(line.attrib.get(key, 'empty'))
        csvwriter.writerow(lineValues)
            

def main():
    parsexml('./Users.xml', 'users.csv')
    print('users done')
    parsexml('./PostHistory.xml', 'posthistory.csv')
    print('post history done')
    
if __name__ == "__main__":
    main()

users done
post history done


From the Users.xml file, find all users which are from Georgia and output to screen their
DisplayName only. (In this case I'm loading the data from the Users csv created above)

In [25]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import re

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("assignment2") \
    .getOrCreate()

rdd = spark.read.csv('./users.csv', header=True).rdd
people = rdd.map(lambda x: (x[3],x[6]))
people = people.filter(lambda x: x[1]!=None)
people = people.filter(lambda x: x[1]!='empty')
people = people.map(lambda x: (x[0],x[1].lower()))
full = people.filter(lambda data: "georgia" in data[1])
full = full.filter(lambda data: "tbilisi" not in data[1])
full = full.filter(lambda data: "rustavi" not in data[1])
abbrev = people.filter(lambda data: re.findall('\\bga\\b',data[1]))
georgiaPeople = full.union(abbrev)
georgiaPeople = georgiaPeople.map(lambda x: x[0])
georgiaPeople = georgiaPeople.collect()

print("These are the names of everyone that lives in Georgia. This does not include the country of Georgia:\n")
print(*georgiaPeople, sep=', ')

These are the names of everyone that lives in Georgia. This does not include the country of Georgia:

Ayush, vkb, Dan Anton, Manish Ranjan, Dato Janez, Neuromeda, Jenna Kwon, Jeff, Jesse Scherer, Melinda Weathers, N.C.W., Tae-Sung Shin, Clark Lin, alerera, Max Boonbandansook, Tony Boyles, pkerl, Nick Larsen, gfritz, Aleksandr Blekh, Michael, azoorob, ontek, Aravind R. Yarram, ilya, Daisuke Aramaki, tempusfugit, Henry Crutcher, Goddard, Matt Simpson, Peter Woolfitt, matt biskup, Jason W, Peter Mourfield, Magsol, Bob Baxley, badjr, mplunney, YC Hu, ryan, Patrick Gerbes, Ilya Lapitan, pradyumnad, Psidom, Teresa Madsen, Brandon, jpm, Mr. Rooter of Savannah, Mr. Rooter of Southeast GA, Khiem Ha, Ahmet Cecen, Guy Gordon, C3Theo, niru dyogi, Vinitha Palani, Mac18, Andrew, Aditya Gogoi, turtlemonvh, Lewis Rodgers, Tarun Luthra, Devendra Lattu, cosmosa, Todd Dawson, ironv, Mboolean, Jimd, David F, PSInf, Chirag, Sandeep Gunda, hellofanengineer, Will Gao, Oriol Mirosa, Andrew King, David, Nghia.

Using the Users.xml file, provide the count for all users which joined (CreationDate) in 2017. (30
points). Output this to the screen

In [23]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("assignment2") \
    .getOrCreate()

rdd = spark.read.csv('./users.csv', header=True).rdd
dates = rdd.map(lambda x: (x[2],1))
dates = dates.filter(lambda x: x[0] != None)
dates = dates.filter(lambda x: "2017" in x[0])
dates = dates.filter(lambda x: x[0][0].isdigit())
dates = dates.map(lambda x: (x[0][0:4], 1))
dates = dates.reduceByKey(lambda x,y: x + y)
count = dates.collect()

print("The number of people that created their account in 2017 is: " + str(count[0][1]))

The number of people that created their account in 2017 is: 14239


Using the PostHistory file, count the number of Posts that feature the words “Spark” and
“Scala”. Output this to the screen.

In [12]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import re

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("assignment2") \
    .getOrCreate()

rdd = spark.read.csv('./posthistory.csv', header=True).rdd
text = rdd.map(lambda x: x[6])
text = text.filter(lambda x: x != None)
text = text.map(lambda x: x.lower())
sparkText = text.filter(lambda line: re.findall('\\bspark\\b', line))
scalaText = text.filter(lambda line: re.findall('\\bscala\\b', line))

print("Total number of posts that feature word 'scala': " + str(scalaText.count()))
print("Total number of posts that feature word 'spark': " + str(sparkText.count()))

both = sparkText.intersection(scalaText).count()

print("Total number of posts that feature both words: " + str(both))

Total number of posts that feature word 'scala': 124
Total number of posts that feature word 'spark': 739
Total number of posts that feature both words: 52


Using the PostHistory file, provide a total count of the words used by each distinct user. In other
words, count all words in all posts for each user and display this to screen. You can only identify
users by the UserID 

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import re

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("assignment2") \
    .getOrCreate()

rdd = spark.read.csv('./posthistory.csv', header=True).rdd
wordT = rdd.map(lambda x: (x[5], x[6]))
wordT = wordT.filter(lambda x: x[1] != None)
wordT = wordT.map(lambda x: (x[0], x[1].lower()))
wordT = wordT.map(lambda x: (x[0], x[1].split(" ")))
